In [2]:
# from hdfs import InsecureClient
# import dask.dataframe as dd
# import pandas as pd
# import numpy as np
# import os
# import posixpath as psp
# import datetime
# import xgboost as xgb
# import csv
# client = InsecureClient('http://t2ru-bda-mnp-003:14000', user='alexander.rass')
# PATH = r'/user/hive/warehouse/uat_dm.db/ar_rbt_goodok_march_full_cl_active_p/'

# # fpaths = [
# #   psp.join(dpath, fname)
# #   for dpath, _, fnames in client.walk(PATH)
# #   for fname in fnames
# # ]
# # for i, path in enumerate(fpaths):
# #     print(i)
# #     client.download(path, 'data/'+ '_' + str(i)+'.parquet')

# files = client.list(PATH, status=True)
# for file_name, file_info in files:
#     if file_info['type'] == 'FILE':
#         path = PATH + file_name
#         print(client.status(path))
#         client.download(path, 'data/')

# В следующей клетке нужно раскомментить вторую строчку чтобы читать все файлы, первая сделана для быстрого примера

In [1]:
from hdfs import InsecureClient
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
import posixpath as psp
import datetime
import xgboost as xgb
import csv
#filename = os.path.join('data', '*1.parquet')
filename = os.path.join('data', '*.parq')

In [2]:
from dask.distributed import Client
c = Client(n_workers = 8)
c

Client Scheduler: tcp://127.0.0.1:35501 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 56 Memory: 270.18 GB


In [3]:
df = dd.read_parquet(path=filename, columns = ['report_date', 'dest_number', 'first_seen', 'last_voice_out', 'last_voice_in'
, 'last_msg_out', 'last_msg_in', 'cnt_voice_out', 'cnt_voice_in', 'cnt_msg_out', 'cnt_msg_in'], index = 'subs_id')
#df.index = df.index.astype(np.int64)
df['dest_number'] = df['dest_number'].astype('category')
df['lifetime'] = (df['report_date'] - df['first_seen']).dt.days
df['last_contact'] = df.loc[:, ['last_voice_in', 'last_voice_out'
                                , 'last_msg_in', 'last_msg_out']].min(axis=1)
df['freq_voice'] = (df['cnt_voice_out']+df['cnt_voice_in'])/(df['lifetime']+1)
df['freq_msg'] = (df['cnt_msg_out']+df['cnt_msg_in'])/(df['lifetime']+1)
df['voice_ratio'] = df['cnt_voice_out']/df['cnt_voice_in']
df['msg_ratio'] = df['cnt_msg_out']/df['cnt_msg_in']
df.drop(axis=1, labels = ['last_voice_in', 'last_voice_out', 'report_date'
                                , 'last_msg_in', 'last_msg_out'])
df = c.persist(df)

In [4]:
af = {'cnt_voice_out':'sum'
     ,'cnt_voice_in':'sum'
     ,'cnt_msg_out':'sum'
     ,'cnt_msg_in':'sum'
     ,'dest_number':'count'
      ,'freq_voice':'mean'
      ,'freq_msg':'mean'
      ,'voice_ratio':'mean'
      ,'msg_ratio':'mean'
     ,'lifetime':['max', 'min']
     ,'last_contact':['min', 'mean']}
cl_agg_all = df.groupby(['subs_id']).agg(af)
cl_agg_all.columns = [' '.join(col).strip() for col in cl_agg_all.columns.values]
cl_agg_all = c.persist(cl_agg_all)

In [5]:
%%time
features_base = ['dest_number count', 'voice_ratio mean', 'freq_msg mean',
       'lifetime max', 'lifetime min', 'cnt_voice_out sum', 'cnt_msg_out sum',
       'freq_voice mean', 'cnt_msg_in sum', 'cnt_voice_in sum',
       'msg_ratio mean', 'last_contact min', 'last_contact mean']

freq = [
b'OKru',b'DpyrBokpyr',b'Google',b'4446',b'MMS',b'4449',b'5557',b'QIWIWallet',b'MailRu',b'WhatsApp',b'WEBBANKIR',
b'Bistrodengi',b'Youla',b'OTTService',b'Access',b'159',b'Avito',b'SBERBANK',b'545',b'78007072279',b'ekapusta',
b'SMSfinance',b'MSG',b'Yandex',b'sangistil',b'78005050400',b'mili.ru',b'597',b'15400',b'73422700006',b'LICARD',
b'Zaymer.ru',b'CLUB-LUKOIL',b'78001000800',b'moneyman',b'tabor.ru',b'gogomusic.net',b'Denga',b'VashiDengi',
b'VIVADENGI',b'centrofin',b'SMS',b'MigCredit',b'aptOtSklada',b'ICQ',b'Viber',b'Chitaensbit',b'74957923097',
b'74957923096',b'Medlab.',b'OTKRITIE',b'pecom.ru',b'74992905223',b'r-ulybka.ru',b'Partner',b'RGS',b'Facebook',b'Aeroflot'
]

df_cast = df[df['dest_number'].isin(freq)]
df_cast['dest_number'] = df_cast['dest_number'].cat.set_categories(freq)
df_cast = df_cast.pivot_table(index='subs_id', columns='dest_number',values='last_contact')
df_cast = cl_agg_all.merge(df_cast, left_index=True, right_index=True, how = 'left')
dt = df_cast.compute()
#c.cancel(df)

CPU times: user 1min 59s, sys: 36.2 s, total: 2min 35s
Wall time: 12min 51s


In [34]:
#dt.columns[dt.columns.isin(freq)] = dt.columns[dt.columns.isin(freq)].str.decode('utf-8')
#t.columns
dt.columns = features_base+[f.decode('utf-8') for f in freq]

In [42]:
label = dd.read_csv('label.csv').compute()
label.sort_values(by= ['subs_id', 'label'], inplace=True)
label.drop_duplicates(subset=['subs_id'], inplace =True)
label = label.set_index('subs_id')
train = dt.merge(label, how = 'left', left_index=True, right_index=True)
train.to_parquet('train.parquet')

In [59]:
booster = xgb.Booster()
booster.load_model('rbt_xgb.xgb')

In [62]:
features = dt.columns
dtest = xgb.DMatrix(data = dt[features].values, feature_names = features
                    , missing = np.nan)


In [63]:
dt['pred'] = booster.predict(dtest)